In [159]:
import pandas as pd
import numpy as np
import plotly.express as px

In [160]:
ORCHESTRATORS_ORDER = ["AIRFLOW", "ARGO WORKFLOW", "WINDMILL", "TEMPORAL"]

EXPERIMENT = "scenar_2a_4X4"

#EXPERIMENT = "nothing_"

METRICS_LIST = [
    "cpu_mean",
    "cpu_max",
    "memory_mean",
    "memory_max",
    "sched_time_mean",
    "sched_time_max",
    "exec_time_mean",
    "exec_time_max"]

VISU_STACK_OPTIONS = {
    "cpu_mean" : True,
    "cpu_max" : True,
    "memory_mean" : True,
    "memory_max" : True,
    "sched_time_mean" : False,
    "sched_time_max" : False,
    "exec_time_mean": False,
    "exec_time_max" : False
}

In [161]:
def plot_chart(long, metric, log_threshold: int = 50):
    
    fig = px.bar(
            long,
            x="orchestrator",
            y="value",
            color="experiment",
            title=f"{metric} usage by orchestrator",
            color_discrete_sequence=["#277bab", "#5ABC98"]
        )     

    fig.update_layout(
            barmode="stack",
            yaxis_title=f"{metric} on period",
            font=dict(size=16),
            legend=dict(
                title="Component",
                orientation="v",
                x=1.02,
                y=0.5,
                font=dict(size=18),
            ),
            xaxis_title=""
        )

    if long['value'].max() / long['value'].min() > log_threshold:
            fig.update_yaxes(type="log")
            fig.update_layout(title=f"{metric} usage by orchestrator - LOG SCALE")

    fig.show()

In [162]:
def plot_metric(df: pd.DataFrame, metric: str, 
                stack_option: bool = True, 
                log_threshold: int = 50) -> None:
    """
    Plot the specified metric for different orchestrators using Plotly.
    """

    # Keep only CPU metrics for now
    df_red = df[df["experiment"].str.contains(metric)].copy()



    if len(df_red) == 0:
        print(f"No data for metric: {metric}")
        return
    

    # Long format: orchestrator / value
    long = df_red.melt(
        id_vars="experiment",
        var_name="orchestrator",
        value_name="value"
    )


    long['value'] = long['value'].clip(lower=1e-3)

    if stack_option:

        plot_chart(long, metric=metric, log_threshold=log_threshold)

        print("-" * 40)

    else:

        for exp in long['experiment'].unique():

            long_red = long.loc[long['experiment'] == exp, :]


            plot_chart(long_red, metric=exp, log_threshold=log_threshold)

            print("-" * 40)


In [163]:
def experiment_parsing(df: pd.DataFrame, experiment: str) -> pd.DataFrame:
    """
    Return rows where the 'experiment' column starts with the provided experiment string.
    Handles missing values safely and returns a copy with reset index.
    """
    df_res = df.__deepcopy__()

    if not isinstance(experiment, str) or experiment == "":
        return df_res.copy()

    mask = df_res["experiment"].fillna("").astype(str).str.startswith(experiment)

    df_red = df_res.loc[mask].reset_index(drop=True)

    df_red['experiment'] = df_red['experiment'].apply(lambda x: x.split(experiment)[1])

    df_red = df_red[['experiment'] + ORCHESTRATORS_ORDER]


    # additionnal cast
    for orch in ORCHESTRATORS_ORDER:
        df_red[orch] = df_red[orch].apply(lambda v: float(v))

    return df_red

--------------

In [164]:
df_results = pd.read_csv("result/bench_raw_results_2.csv")

df_results.rename(columns={"Unnamed: 0": "experiment"}, inplace=True)
df_re = experiment_parsing(df_results, EXPERIMENT)
df_re

,experiment,AIRFLOW,ARGO WORKFLOW,WINDMILL,TEMPORAL
0,_perm_cpu_mean,1.40,0.02,0.173,0.640
1,_perm_cpu_max,2.51,0.06,0.220,0.760
2,_perm_memory_mean,5990.00,396.00,825.000,2020.000
3,_perm_memory_max,6300.00,404.00,828.000,2054.000
4,_work_cpu_mean,0.72,0.07,0.220,0.125
5,_work_cpu_max,1.88,0.31,1.170,0.370
6,_work_memory_mean,4755.00,1200.00,325.000,392.000
7,_work_memory_max,6169.00,2092.00,473.000,491.000
8,_sched_time_mean,7.12,35.20,1.780,11.100
9,_sched_time_max_f,4.93,201.00,1.740,48.500


In [165]:
for metric in METRICS_LIST:

    stack_option = VISU_STACK_OPTIONS.get(metric, True)

    plot_metric(df_re, metric, 
                    stack_option=stack_option, 
                    log_threshold=10e6)

----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------
No data for metric: exec_time_max
